In [ ]:
source extend_bashkernel-2modes.source

# This cell should be hidden and execute automatically when the page loads

# Make my-script the default when notebook is loaded:
set-global-evaluation-mode "my-script"

To set the notebook mode, choose "my-script" or "provided-script" and run the cell.

In [ ]:
set-global-evaluation-mode "provided-script"

# Instructions

## Overview: objectives, tasks, sections

This notebook has two main objectives.  The first is to install Jenkins inside a Wakame-vdc instance.  The second is to install plugins into Jenkins that will be needed for subsequent notebooks.

In the notebook sections below, each objective is split into a number of small tasks that must be done in-order.  Each task has three sections:

  * **task details**: This section explains exactly what needs to be done.  Sometimes the task can be done from the command line, therefore an empty **"my-script cell"** is provided here, and your goal is to write a script in the my-script cell and then run the cell to do the task. In other cases, the task needs to be done in a graphical user interface, so your goal is to open separate web page, connect to the appropriate web GUI, and do the task there.
  
  * **examples**: This section is divided into sub-sections that give examples or hints about how to do the task, if you need them.  In most cases there is a special **provided-script** section that has a cell that can be evaluated to do the task automatically.  When there is a my-script cell for the task, lines appear in the provided-script that can be copied to the my-script cell to do the task.  Note that you should only look at the examples one-by-one and only after you have tried to do the task without looking at the next hint.  Copying and pasting from the provided-script should only be done if time does not allow for you to come up with your own solution.
    
  * **check**: This section contains a cell that can be run to automatically check that the task has been done.  You should run this cell and confirm success before moving on to the next task.
 
Sometimes the **task details** and **examples** sections are replaced by a **template** section. Such templates are used to simply later tasks that are too complicated to be done in a reasonable amount of time.


## Notebook modes: my-script and provided-script

Jupyter notebooks have a "Run All Above" feature that can be used with this notebook.  In order for this to work, it is important that only my-script or provided-script cells are run, but not both.  The cell at the top of notebook is used to selecting the mode.

This mode is also in effect when you run cells individually.  Therefore, the mode must be changed to my-script before running the scripts you create in the cells below.

# Objective #1: Install Jenkins in a Wakame-vdc instance

## Task: Reset Wakame-vdc

### Task Details

The first task is to terminate any Wakame-vdc instances that are currently running.  (Doing this task is not necessary if this Jupyter notebook has not been run yet and no instances are currently running.)  Connect to the Wakame-vdc Web GUI, check if there are any instances running, and terminate them, if so.

### Examples

<a href="https://cloud.githubusercontent.com/assets/9693694/13846358/d09c4b80-ec89-11e5-90da-c3b216619f7b.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846358/d09c4b80-ec89-11e5-90da-c3b216619f7b.png" width=15% height=15% align="left"></a>

#### Provided script

In [ ]:
# Task: introduction-terminate-wakame-instances
# Evaluation mode: provided-script

# Here is an automated alternative to using the Wakame-vdc GUI.
# The following Bash code will delete all Wakame-vdc instances by using
# the output from "mussel instance index" to get a list of instance IDs,
# and then using "mussel instance destroy" to terminate each ID.
# These mussel commands can also be used in an interactive terminal.

check_instances()
{
    mussel instance index | \
        (
            date=xxx # only instances database rows with empty "deleted_at" should be displayed
            while read ln; do
                case "$ln" in
                    *:id:*)
                        [ "$date" = "" ] && echo "$out"
                        read dash label theid <<<"$ln"
                        out="     $theid "
                        ;;
                    *:deleted_at:*)
                        read label date rest <<<"$ln"
                        ;;
                esac
            done
            [ "$date" = "" ] && echo "$out"
        )

}

ilist="$(check_instances 2>/dev/null)"
for inst in $ilist; do
    echo "Terminating instance: $inst"
    mussel instance destroy $inst 2>/dev/null
done

# This sleep is necessary so Wakame-vdc will reuse the 10.0.2.100 IP
# addresss.
sleep 15  # TODO: remove the need for this


### Check

In [ ]:
# Task: introduction-terminate-wakame-instances
# Evaluation mode: load

# This cell should be hidden.

# copy/paste of the provided-script code:
check_instances()
{
    mussel instance index | \
        (
            date=xxx # only instances database rows with empty "deleted_at" should be displayed
            while read ln; do
                case "$ln" in
                    *:id:*)
                        [ "$date" = "" ] && echo "$out"
                        read dash label theid <<<"$ln"
                        out="     $theid "
                        ;;
                    *:deleted_at:*)
                        read label date rest <<<"$ln"
                        ;;
                esac
            done
            [ "$date" = "" ] && echo "$out"
        )

}

ilist="$(check_instances 2>/dev/null)"
for inst in $ilist; do
    echo "Terminating instance: $inst"
    mussel instance destroy $inst 2>/dev/null
done

# This sleep is necessary so Wakame-vdc will reuse the 10.0.2.100 IP
# addresss.
sleep 15  # TODO: remove the need for this


In [ ]:
# Task: introduction-terminate-wakame-instances
# Evaluation Mode: check


## Task: Start up Wakame-vdc Instance

### Task Details

Using the Wakame-vdc Web GUI, start a Wakame-vdc instance we can start virtual instances from a machine image that we can work on.

Start an instance using **wmi-centos1d64** as the machine image. Choose **vz.xlarge** for the instance spec, and **ssh-cicddemo** and **sg-cicddemo** for the SSH key pair and Security group.  Connect eth0 to the **demo1** network.

### Examples

<a href="https://cloud.githubusercontent.com/assets/9693694/13846360/d3564506-ec89-11e5-843f-f3a71d77e76f.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846360/d3564506-ec89-11e5-843f-f3a71d77e76f.png" height="15%" width="15%" align="left"></a>
<a href="https://cloud.githubusercontent.com/assets/9693694/13846362/d7255e74-ec89-11e5-993a-682f15e14523.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846362/d7255e74-ec89-11e5-993a-682f15e14523.png" height="15%" width="15%" align="left"></a>
<a href="https://cloud.githubusercontent.com/assets/9693694/13846365/da32cf8e-ec89-11e5-9085-c9431a07362f.png"><img src="https://cloud.githubusercontent.com/assets/9693694/13846365/da32cf8e-ec89-11e5-9085-c9431a07362f.png" height="15%" width="15%" align="left"></a>

#### Provided script

In [ ]:
# Task: introduction-launch-wakame-instance
# Evaluation Mode: provided-script

# Here is an automated alternative to using the Wakame-vdc GUI.
# The following Bash code will launch a Wakame-vdc instance by using
# "mussel instance create".

cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

mussel instance create --cpu-cores 2 --hypervisor openvz \
    --image-id wmi-centos1d64 --memory-size 2048 \
    --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos

### Check

In [ ]:
# Task: introduction-launch-wakame-instance
# Evaluation Mode: load

# This cell should be hidden.

cat <<EOS >vifs.json
{
 "eth0":{"network":"nw-demo1","security_groups":"sg-cicddemo"}
}
EOS

mussel instance create --cpu-cores 2 --hypervisor openvz \
    --image-id wmi-centos1d64 --memory-size 2048 \
    --ssh-key-id ssh-cicddemo --vifs vifs.json --display-name centos

In [ ]:
# Task: introduction-launch-wakame-instance
# Evaluation Mode: check

## Task: Wait for ssh to be ready

### Task Details

After confirming that the instance is running in the Wakame-vdc Web GUI, use the ssh command in the following cell to confirm that the instance is running correctly.  Be sure to confirm that the IP address matches the one assigned to the instance by Wakame-vdc.  ((As a bonus, it would be good to be able to write a script that actually waits for the instanace and its ssh server to become active, since that technique is necessary for later scripts.))

In [ ]:
# Enter and run your commands and scripts in this cell:



### Examples

#### Provided script

In [ ]:
# Task: introduction-wait-for-ssh
# Evaluation Mode: provided-script

# The first part of this script waits for the ssh port (#22) to respond.
# At the end, an ssh connection is attempted.

: ${IP:=10.0.2.100}
while [[ "$(echo | nc -w 1 "$IP" 22)" != *SSH* ]]; do
  sleep 2
  echo "Waiting on SSH at $IP..."
done
echo "SSH is active at $IP, port 22"

ssh -qi ../mykeypair root@10.0.2.100 uptime

### Check

In [ ]:
# Task: introduction-wait-for-ssh
# Evaluation Mode: load

# This script should be hidden.

# It is also a bit awkward because the user's task is to check with ssh, but
# really waiting for ssh is necessary.  Should the user learn how to write
# a script that waits here?  If so how can it be checked?  ReRun the notebook.
# Hmmm, maybe that would be a nice demo.

: ${IP:=10.0.2.100}
while [[ "$(echo | nc -w 1 "$IP" 22)" != *SSH* ]]; do
  sleep 2
  echo "Waiting on SSH at $IP..."
done
echo "SSH is active at $IP, port 22"

ssh -qi ../mykeypair root@10.0.2.100 uptime

In [ ]:
# Task: introduction-wait-for-ssh
# Evaluation Mode: check

# Objective #2: Install Packages

## Task: repoファイルのインストール

### Task Details

jenkins用yumリポジトリを追加します。 Download the file at http://pkg.jenkins-ci.org/redhat/jenkins.repo and put it at /etc/yum.repos.d/jenkins.repo inside the instance.

In [ ]:
# Evaluation Mode: myscript


### Examples

ssh -i ../mykeypair root@10.0.2.100 curl -fSkL some-url -o some-directory

#### Provided script

In [ ]:
# Task: introduction-jenkins-repo-file
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}
ssh -qi ../mykeypair root@$IP curl -fSkL http://pkg.jenkins-ci.org/redhat/jenkins.repo -o /etc/yum.repos.d/jenkins.repo

### Check

In [ ]:
# Task: introduction-jenkins-repo-file
# Evaluation Mode: check


## Task: 公開鍵のインポート

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-import-public-key
# Evaluation Mode: provided-script

: ${IP:=10.0.2.100}
ssh -qi ../mykeypair root@$IP rpm --import http://pkg.jenkins-ci.org/redhat/jenkins-ci.org.key

### Check

In [ ]:
# Task: introduction-import-public-key
# Evaluation Mode: check


## Task: java実行環境のインストール

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-install-java
# Evaluation Mode: provided-script

# This installs the Oracle version of Java.  The RPM file was
# already downloaded to /home/centos/notebooks/downloads

: ${IP:=10.0.2.100}
cd /home/centos/notebooks/.downloads
tar c jdk-8u73-linux-x64.rpm | ssh -qi /home/centos/mykeypair root@$IP tar xv

ssh -qi /home/centos/mykeypair root@$IP <<EOS

rpm -ivh jdk-8u73-linux-x64.rpm

cat <<CFG >/etc/profile.d/java.sh
JAVA_HOME=/usr/java/jdk1.8.0_25/
PATH=$JAVA_HOME/bin:$PATH
export PATH JAVA_HOME
export CLASSPATH=.
CFG

EOS

### Check

In [ ]:
# Task: introduction-install-java
# Evaluation Mode: check


## Task: jenkins coreのインストール

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-install-jenkins
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}
ssh -qi /home/centos/mykeypair root@$IP <<EOS

yum install -y jenkins

EOS

### Check

In [ ]:
# Task: introduction-install-jenkins
# Evaluation Mode: check

## Task: 関連するビルドツールのインストール

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-install-others
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}

ssh -qi /home/centos/mykeypair root@$IP <<'EOS'

yum install -y git iputils nc qemu-img parted kpartx rpm-build automake createrepo openssl-devel zlib-devel readline-devel gcc

EOS

### Check

In [ ]:
# Task: introduction-install-others
# Evaluation Mode: check

# Objective #3: Configuration and Start Jenkins

## Task: sudo権限の付与

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-sudo-config
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}

ssh -qi /home/centos/mykeypair root@$IP <<'EOS' 2>/dev/null

echo 'jenkins ALL=(ALL) NOPASSWD: ALL' >>/etc/sudoers

EOS

### Check

In [ ]:
# Task: introduction-sudo-config
# Evaluation Mode: check

## Task: tty設定の変更

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-sudo-notty
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}

ssh -qi /home/centos/mykeypair root@$IP <<'EOS' 2>/dev/null

sed -i "s/^\(^Defaults\s*requiretty\).*/# \1/" /etc/sudoers

EOS

### Check

In [ ]:
# Task: introduction-sudo-notty
# Evaluation Mode: check

## Task: Starting jenkins

### Task Details

In [ ]:
# Evaluation Mode: myscript


### Examples

#### Provided script

In [ ]:
# Task: introduction-start-service
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}

ssh -qi /home/centos/mykeypair root@$IP <<'EOS' 2>/dev/null

service jenkins start

EOS

### Check

In [ ]:
# Task: introduction-start-service
# Evaluation Mode: check

# Objective #4: Install plugins into Jenkins

## Task: Install plugins into Jenkins

### Task Details

In order to build our enviroment and make better use of jenkins we need to install certain plugins. 

Install the following plugins and then restart jenkins.
* git
* git-client
* hipchat
* rbenv
* parameterized-trigger

### Examples

#### Provided script

In [ ]:
# Task: introduction-install-jenkins-plugins
# Evaluation Mode: provided-script
: ${IP:=10.0.2.100}

ssh -qi /home/centos/mykeypair root@${IP} <<'EOS' 2> /dev/null

# Installs jenkins cli tool to make calls through terminal
curl -O http://localhost:8080/jnlpJars/jenkins-cli.jar

# Installs the plugins
java -jar jenkins-cli.jar -s http://localhost:8080 install-plugin rbenv hipchat git git-client parameterized-trigger

service jenkins restart
EOS

### Check

## Task: Make sure jenkins is up and running

### Task Details

### Examples

#### Provided script

In [ ]:
: ${IP:=10.0.2.100}

ssh -qi /home/centos/mykeypair root@${IP} <<'EOS' 2> /dev/null

while ! curl -I -s http://localhost:8080/ | grep -q "200 OK" ; do
    echo "Waiting for jenkins..."
    sleep 3
done

echo "Jenkins is ready."

EOS

### Check

## Task: Configure the git plugin